In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from prophet import Prophet
from datetime import datetime
import glob

In [ ]:
#se ejecuta un ciclo para leer todos los archivos csv que contienen la informacion de consumo de los clientes
csv_files = glob.glob('*.csv')

# dataframe vacio para almacenar los datos
SB = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df['fuente']= csv_file
    SB = pd.concat([SB, df])

In [9]:
SB.loc[SB.Active_energy<0, 'Active_energy']=0
SB.loc[SB.Reactive_energy<0, 'Reactive_energy']=0
SB.loc[SB.Voltaje_FA<0, 'Voltaje_FA']=0
SB.loc[SB.Voltaje_FC<0, 'Voltaje_FC']=0

In [10]:
SB

,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,fuente
0,2021-01-01 00:00:00,0.357841,0.282788,455.139171,510.561002,DATOSCLIENTE1.csv
1,2021-01-01 01:00:00,0.372264,0.431377,469.978787,469.917178,DATOSCLIENTE1.csv
2,2021-01-01 02:00:00,1.044687,0.338626,468.721120,546.949147,DATOSCLIENTE1.csv
3,2021-01-01 03:00:00,0.566425,0.495791,452.329255,444.122989,DATOSCLIENTE1.csv
4,2021-01-01 04:00:00,1.080556,0.472018,513.477596,535.463719,DATOSCLIENTE1.csv
...,...,...,...,...,...,...
11410,2022-04-21 10:00:00,0.960105,0.473234,1273.150602,1027.084539,DATOSCLIENTE30.csv
11411,2022-04-21 11:00:00,0.624300,0.699936,1063.524968,1205.829819,DATOSCLIENTE30.csv
11412,2022-04-21 12:00:00,0.985633,0.123560,1207.284283,1127.893714,DATOSCLIENTE30.csv
11413,2022-04-21 13:00:00,0.710436,0.399262,1205.012971,1090.835898,DATOSCLIENTE30.csv


In [11]:
SB['Fecha'] = pd.to_datetime(SB['Fecha'])
SB['Year'] = SB['Fecha'].dt.year
SB['Month'] = SB['Fecha'].dt.month
SB['Day'] = SB['Fecha'].dt.day
SB['DayWeek'] = SB['Fecha'].dt.day_of_week
SB['Hour'] = SB['Fecha'].dt.hour
SB

,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,fuente,Year,Month,Day,DayWeek,Hour
0,2021-01-01 00:00:00,0.357841,0.282788,455.139171,510.561002,DATOSCLIENTE1.csv,2021,1,1,4,0
1,2021-01-01 01:00:00,0.372264,0.431377,469.978787,469.917178,DATOSCLIENTE1.csv,2021,1,1,4,1
2,2021-01-01 02:00:00,1.044687,0.338626,468.721120,546.949147,DATOSCLIENTE1.csv,2021,1,1,4,2
3,2021-01-01 03:00:00,0.566425,0.495791,452.329255,444.122989,DATOSCLIENTE1.csv,2021,1,1,4,3
4,2021-01-01 04:00:00,1.080556,0.472018,513.477596,535.463719,DATOSCLIENTE1.csv,2021,1,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...
11410,2022-04-21 10:00:00,0.960105,0.473234,1273.150602,1027.084539,DATOSCLIENTE30.csv,2022,4,21,3,10
11411,2022-04-21 11:00:00,0.624300,0.699936,1063.524968,1205.829819,DATOSCLIENTE30.csv,2022,4,21,3,11
11412,2022-04-21 12:00:00,0.985633,0.123560,1207.284283,1127.893714,DATOSCLIENTE30.csv,2022,4,21,3,12
11413,2022-04-21 13:00:00,0.710436,0.399262,1205.012971,1090.835898,DATOSCLIENTE30.csv,2022,4,21,3,13


In [12]:
SB.describe()

,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Year,Month,Day,DayWeek,Hour
count,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000
mean,1.472884,0.873086,1420.188470,1438.515836,2021.500912,5.955615,15.607762,3.001746,11.497209
std,1.717838,1.158846,766.299118,746.447449,0.621279,3.519334,8.783858,1.999393,6.922158
min,0.000000,0.000000,0.031000,0.031000,2021.000000,1.000000,1.000000,0.000000,0.000000
25%,0.242788,0.112832,719.462874,748.148254,2021.000000,3.000000,8.000000,1.000000,5.000000
50%,0.810771,0.380650,1625.493463,1634.696089,2021.000000,6.000000,16.000000,3.000000,11.000000
75%,1.992488,1.222834,2037.276385,2040.554497,2022.000000,9.000000,23.000000,5.000000,17.000000
max,14.622644,11.135141,4266.229746,4399.038932,2023.000000,12.000000,31.000000,6.000000,23.000000


AGRUPACIÓN POR AÑO Y MES

In [10]:
SB_Agrupado_AM = SB.groupby(["fuente", "Year","Month"])[["fuente", "Year","Month","Active_energy","Reactive_energy","Voltaje_FA","Voltaje_FC"]].agg({
    'fuente':'first',
    'Year':'first',
    'Month':'first',
    "Active_energy":'sum',
    "Reactive_energy":'sum',
    "Voltaje_FA":'sum',
    "Voltaje_FC":'sum'
    }
)
SB_Agrupado_AM.describe()

,Year,Month,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
count,656.000000,656.000000,656.000000,656.000000,6.560000e+02,6.560000e+02
mean,2021.535061,5.867378,1040.505239,616.783293,1.003279e+06,1.016226e+06
std,0.645825,3.502989,1068.576664,701.765092,5.097992e+05,4.985636e+05
min,2021.000000,1.000000,0.000000,0.000000,2.083200e+01,3.495794e+02
25%,2021.000000,3.000000,326.825030,119.752483,5.266159e+05,5.399684e+05
50%,2021.000000,5.000000,611.005026,343.751221,1.106712e+06,1.128775e+06
75%,2022.000000,9.000000,1487.656658,938.920272,1.451994e+06,1.453097e+06
max,2023.000000,12.000000,5975.622323,3382.717277,2.368473e+06,2.367976e+06


In [11]:
SB_Agrupado_AM = SB_Agrupado_AM[~((SB_Agrupado_AM['Year']==2023) & (SB_Agrupado_AM['Month']==4))]

In [13]:
SB_Agrupado_AM=SB_Agrupado_AM.reset_index(drop=True)

In [14]:
def identify_outliers(group, threshold=3):
    mean = group['Active_energy'].mean()
    std_dev = group['Active_energy'].std()
    group['Outlier_Active_energy'] = group['Active_energy'].apply(lambda x: 1 if np.abs((x - mean) / std_dev) > threshold else 0)
    
    mean = group['Reactive_energy'].mean()
    std_dev = group['Reactive_energy'].std()
    group['Outlier_Reactive_energy'] = group['Reactive_energy'].apply(lambda x: 1 if np.abs((x - mean) / std_dev) > threshold else 0)
    
    mean = group['Voltaje_FA'].mean()
    std_dev = group['Voltaje_FA'].std()
    group['Outlier_Voltaje_FA'] = group['Voltaje_FA'].apply(lambda x: 1 if np.abs((x - mean) / std_dev) > threshold else 0)
    
    mean = group['Voltaje_FC'].mean()
    std_dev = group['Voltaje_FC'].std()
    group['Outlier_Voltaje_FC'] = group['Voltaje_FC'].apply(lambda x: 1 if np.abs((x - mean) / std_dev) > threshold else 0)
    
    return group

# Apply the function to each group
outlier_df_AM = SB_Agrupado_AM.groupby('fuente').apply(identify_outliers)

In [15]:
outlier_df_AM[outlier_df_AM['fuente']=='DATOSCLIENTE11.csv']

fuente  Year  Month  Active_energy  \
fuente                                                                  
DATOSCLIENTE11.csv 54  DATOSCLIENTE11.csv  2021      1     418.517678   
                   55  DATOSCLIENTE11.csv  2021      2     382.568269   
                   56  DATOSCLIENTE11.csv  2021      3     410.862320   
                   57  DATOSCLIENTE11.csv  2021      4     406.530230   
                   58  DATOSCLIENTE11.csv  2021      5     409.957139   
                   59  DATOSCLIENTE11.csv  2021      6     411.523122   
                   60  DATOSCLIENTE11.csv  2021      7     423.620566   
                   61  DATOSCLIENTE11.csv  2021      8     420.171115   
                   62  DATOSCLIENTE11.csv  2021      9     404.929724   
                   63  DATOSCLIENTE11.csv  2021     10     522.347347   
                   64  DATOSCLIENTE11.csv  2021     11     778.536997   
                   65  DATOSCLIENTE11.csv  2021     12    1024.552704   
                   66  DATOSCLIENTE11.csv  2022      1     657.003503   
                   67  DATOSCLIENTE11.csv  2022      2     746.289865   
                   68  DATOSCLIENTE11.csv  2022      3     716.418098   
                   69  DATOSCLIENTE11.csv  2022      4     899.308650   
                   70  DATOSCLIENTE11.csv  2022      5     757.550720   
                   71  DATOSCLIENTE11.csv  2022      6     405.679340   
                   72  DATOSCLIENTE11.csv  2022      7     409.860825   
                   73  DATOSCLIENTE11.csv  2022      8      87.378134   

                       Reactive_energy    Voltaje_FA    Voltaje_FC  \
fuente                                                               
DATOSCLIENTE11.csv 54        45.500795  1.436951e+06  1.427436e+06   
                   55        40.030509  1.298547e+06  1.288910e+06   
                   56        45.613205  1.437367e+06  1.427438e+06   
                   57        43.566415  1.390678e+06  1.382959e+06   
                   58        45.991775  1.438968e+06  1.430188e+06   
                   59        42.568212  1.391758e+06  1.380173e+06   
                   60        45.708317  1.438022e+06  1.427299e+06   
                   61        46.119507  1.437118e+06  1.426503e+06   
                   62        44.345135  1.392018e+06  1.382004e+06   
                   63        43.341885  1.437665e+06  1.425468e+06   
                   64        30.464482  1.391751e+06  1.381348e+06   
                   65        32.562061  1.438624e+06  1.427123e+06   
                   66        31.997614  1.436229e+06  1.426970e+06   
                   67        28.227684  1.299636e+06  1.288968e+06   
                   68        38.322777  1.438324e+06  1.427096e+06   
                   69        37.467062  1.392199e+06  1.383377e+06   
                   70        44.223446  1.436693e+06  1.427525e+06   
                   71        42.624404  1.391038e+06  1.380305e+06   
                   72        47.175319  1.437826e+06  1.427573e+06   
                   73        10.230285  2.936399e+05  2.910926e+05   

                       Outlier_Active_energy  Outlier_Reactive_energy  \
fuente                                                                  
DATOSCLIENTE11.csv 54                      0                        0   
                   55                      0                        0   
                   56                      0                        0   
                   57                      0                        0   
                   58                      0                        0   
                   59                      0                        0   
                   60                      0                        0   
                   61                      0                        0   
                   62                      0                        0   
                   63                      0                        0   
           

In [17]:
outlier_df_AM['Anomalia']= (outlier_df_AM['Outlier_Active_energy']+outlier_df_AM['Outlier_Reactive_energy']+outlier_df_AM['Outlier_Voltaje_FA']+outlier_df_AM['Outlier_Voltaje_FC'])
#outlier_df.loc[outlier_df.Anomalia>0, 'Anomalia']=1

In [18]:
outlier_df_AM=outlier_df_AM.reset_index(drop=True)

In [19]:
outlier_df_AM.groupby('fuente')[['Anomalia']].agg("sum").sort_values(by ='Anomalia', ascending=False)

,Anomalia
fuente,
DATOSCLIENTE11.csv,3
DATOSCLIENTE28.csv,3
DATOSCLIENTE2.csv,3
DATOSCLIENTE1.csv,2
DATOSCLIENTE22.csv,2
DATOSCLIENTE5.csv,2
DATOSCLIENTE4.csv,2
DATOSCLIENTE30.csv,2
DATOSCLIENTE3.csv,2


In [24]:
outlier_df_AM['fuente']=outlier_df_AM['fuente'].str.replace('DATOSCLIENTE','').str.replace('.csv','')
outlier_df_AM

,fuente,Year,Month,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Outlier_Active_energy,Outlier_Reactive_energy,Outlier_Voltaje_FA,Outlier_Voltaje_FC,Anomalia
0,1,2021,1,1179.028493,415.713658,3.706093e+05,3.707132e+05,0,0,0,0,0
1,1,2021,2,1490.128945,231.737781,5.016982e+05,5.034547e+05,0,0,0,0,0
2,1,2021,3,1728.240344,978.845693,8.359615e+05,8.364873e+05,0,0,1,1,2
3,1,2021,4,2228.447578,1304.500199,3.592254e+05,3.587905e+05,0,0,0,0,0
4,1,2021,5,2190.919624,1269.936238,3.706615e+05,3.702465e+05,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
641,9,2022,11,620.205229,744.085217,1.406934e+06,1.418035e+06,0,0,0,0,0
642,9,2022,12,1145.982088,1222.681333,1.135527e+06,1.132324e+06,0,0,0,0,0
643,9,2023,1,174.513827,234.626434,1.618204e+06,1.621082e+06,0,0,0,0,0
644,9,2023,2,159.099194,215.007085,1.463043e+06,1.473291e+06,0,0,0,0,0


In [25]:
Anomalias_AM = outlier_df_AM.drop(['Active_energy','Reactive_energy','Voltaje_FA',	'Voltaje_FC',	'Outlier_Active_energy',	'Outlier_Reactive_energy',	'Outlier_Voltaje_FA',	'Outlier_Voltaje_FC'], axis=1)
Anomalias_AM.loc[Anomalias_AM.Anomalia>0, 'Anomalia']=1
Anomalias_AM

,fuente,Year,Month,Anomalia
0,1,2021,1,0
1,1,2021,2,0
2,1,2021,3,1
3,1,2021,4,0
4,1,2021,5,0
...,...,...,...,...
641,9,2022,11,0
642,9,2022,12,0
643,9,2023,1,0
644,9,2023,2,0


In [26]:
datos_AM = outlier_df_AM.drop([ 'Outlier_Active_energy','Outlier_Reactive_energy', 'Outlier_Voltaje_FA', 'Outlier_Voltaje_FC',
       'Anomalia'], axis=1)
datos_AM

,fuente,Year,Month,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
0,1,2021,1,1179.028493,415.713658,3.706093e+05,3.707132e+05
1,1,2021,2,1490.128945,231.737781,5.016982e+05,5.034547e+05
2,1,2021,3,1728.240344,978.845693,8.359615e+05,8.364873e+05
3,1,2021,4,2228.447578,1304.500199,3.592254e+05,3.587905e+05
4,1,2021,5,2190.919624,1269.936238,3.706615e+05,3.702465e+05
...,...,...,...,...,...,...,...
641,9,2022,11,620.205229,744.085217,1.406934e+06,1.418035e+06
642,9,2022,12,1145.982088,1222.681333,1.135527e+06,1.132324e+06
643,9,2023,1,174.513827,234.626434,1.618204e+06,1.621082e+06
644,9,2023,2,159.099194,215.007085,1.463043e+06,1.473291e+06


In [27]:
# Configurando el Isolation Forest Agrupación Año Mes
clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                      max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

# Entrenar el Isolation Forest
clf.fit(datos_AM)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds = clf.predict(datos_AM)

# Añadir las predicciones al DataFrame
datos_AM['anomaly'] = preds


In [30]:
anomalies_AM = datos_AM[datos_AM['anomaly'] == -1]
print(anomalies_AM)

    fuente  Year  Month  Active_energy  Reactive_energy    Voltaje_FA  \
8        1  2021      9    3854.354275      1648.567748  3.582002e+05   
9        1  2021     10    4117.316795      1759.419002  3.704292e+05   
10       1  2021     11    3923.644271      1679.249685  3.595856e+05   
24       1  2023      1     946.243385       491.541213  3.708919e+05   
29      10  2021      3    1686.237575       382.149266  1.978678e+06   
..     ...   ...    ...            ...              ...           ...   
617      8  2023      2     809.963383       319.016009  1.699872e+06   
625      9  2021      7     172.674584       239.194833  1.831292e+06   
631      9  2022      1     173.689647       239.228191  2.109159e+06   
632      9  2022      2     157.994505       211.814933  1.001356e+05   
645      9  2023      3     171.987943       239.717023  4.448266e+05   

       Voltaje_FC  anomaly  
8    3.600851e+05       -1  
9    3.705632e+05       -1  
10   3.588442e+05       -1  
24   3.

In [32]:
anomalies_AM[anomalies_AM['fuente']=='9']

,fuente,Year,Month,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,anomaly
625,9,2021,7,172.674584,239.194833,1.831292e+06,1.825274e+06,-1
631,9,2022,1,173.689647,239.228191,2.109159e+06,2.116245e+06,-1
632,9,2022,2,157.994505,211.814933,1.001356e+05,1.019039e+05,-1
645,9,2023,3,171.987943,239.717023,4.448266e+05,4.568463e+05,-1


In [33]:
Anomalias_IFAM = anomalies_AM.drop(['Active_energy','Reactive_energy','Voltaje_FA',	'Voltaje_FC'], axis=1)

In [34]:
Validacion_AM=Anomalias_AM.merge(Anomalias_IFAM,how='outer')
Validacion_AM['anomaly'] = Validacion_AM['anomaly'].fillna(0)
Validacion_AM


,fuente,Year,Month,Anomalia,anomaly
0,1,2021,1,0,0.0
1,1,2021,2,0,0.0
2,1,2021,3,1,0.0
3,1,2021,4,0,0.0
4,1,2021,5,0,0.0
...,...,...,...,...,...
641,9,2022,11,0,0.0
642,9,2022,12,0,0.0
643,9,2023,1,0,0.0
644,9,2023,2,0,0.0


In [35]:
Validacion_AM['resultado']=Validacion_AM['Anomalia']+Validacion_AM['anomaly']
Validacion_AM.groupby(['fuente','Year', 'Month'])[['Anomalia',	'anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)
Validacion_AM.head()

,fuente,Year,Month,Anomalia,anomaly,resultado
0,1,2021,1,0,0.0,0.0
1,1,2021,2,0,0.0,0.0
2,1,2021,3,1,0.0,1.0
3,1,2021,4,0,0.0,0.0
4,1,2021,5,0,0.0,0.0


In [36]:
tabla_cruzada_AM = pd.crosstab(Validacion_AM['Anomalia'], Validacion_AM['anomaly'])
print(tabla_cruzada_AM)

anomaly   -1.0   0.0
Anomalia            
0           68   550
1           10    18


AGRUPACIÓN POR AÑO MES Y DIA (2)

In [39]:
SB_Agrupado2 = SB.groupby(["fuente", "Year","Month","Day"])[["fuente", "Year","Month","Day","Active_energy","Reactive_energy","Voltaje_FA","Voltaje_FC"]].agg({
    'fuente':'first',
    'Year':'first',
    'Month':'first',
    'Day':'first',
    "Active_energy":'sum',
    "Reactive_energy":'sum',
    "Voltaje_FA":'sum',
    "Voltaje_FC":'sum'
    }
)
SB_Agrupado2.describe()

,Year,Month,Day,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
count,19327.000000,19327.000000,19327.000000,19327.000000,19327.000000,19327.000000,19327.000000
mean,2021.501992,5.954157,15.600766,35.316989,20.934953,34053.440361,34492.896015
std,0.622198,3.518504,8.787520,37.033534,24.271063,18104.388987,17622.591296
min,2021.000000,1.000000,1.000000,0.000000,0.000000,0.744000,0.744000
25%,2021.000000,3.000000,8.000000,10.852845,3.743852,17195.873504,17822.125245
50%,2021.000000,6.000000,16.000000,21.543072,10.994711,38528.293863,39742.819626
75%,2022.000000,9.000000,23.000000,50.895999,34.660911,49248.798665,49251.192855
max,2023.000000,12.000000,31.000000,219.910192,131.713897,87848.013094,88046.824126


In [40]:
SB_Agrupado2 = SB_Agrupado2[~((SB_Agrupado2['Year']==2023) & (SB_Agrupado2['Month']==4))]
SB_Agrupado2=SB_Agrupado2.reset_index(drop=True)

In [47]:
outlier_df2 = SB_Agrupado2.groupby('fuente').apply(identify_outliers)

In [52]:
outlier_df2['Anomalia']= (outlier_df2['Outlier_Active_energy']+outlier_df2['Outlier_Reactive_energy']+outlier_df2['Outlier_Voltaje_FA']+outlier_df2['Outlier_Voltaje_FC'])
outlier_df2=outlier_df2.reset_index(drop=True)
outlier_df2.groupby('fuente')[['Anomalia']].agg("sum").sort_values(by ='Anomalia', ascending=False)

,Anomalia
fuente,
4,128
2,128
3,120
5,76
1,74
7,39
8,34
29,27
11,22


In [65]:
outlier_df2['fuente']=outlier_df2['fuente'].str.replace('DATOSCLIENTE','').str.replace('.csv','')
outlier_df2

,fuente,Year,Month,Day,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Outlier_Active_energy,Outlier_Reactive_energy,Outlier_Voltaje_FA,Outlier_Voltaje_FC,Anomalia
0,1,2021,1,1,29.972064,15.233026,11624.168389,11637.857473,0,0,0,0,0
1,1,2021,1,2,36.978240,18.194945,12208.163321,11936.657816,0,0,0,0,0
2,1,2021,1,3,34.909377,18.749972,12033.759791,12881.126615,0,0,0,0,0
3,1,2021,1,4,36.647375,19.020038,11945.363519,12493.292822,0,0,0,0,0
4,1,2021,1,5,33.051524,17.881993,12020.001492,11668.700439,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19312,9,2023,3,27,5.371967,8.267639,26241.124056,26762.891164,0,0,0,0,0
19313,9,2023,3,28,5.910494,8.670036,25390.551163,25555.142219,0,0,0,0,0
19314,9,2023,3,29,6.022898,8.333407,29584.937317,28882.951142,0,0,0,0,0
19315,9,2023,3,30,5.459185,8.339315,28592.978517,30261.498515,0,0,0,0,0


In [66]:
Anomalias_2 = outlier_df2.drop(['Active_energy','Reactive_energy','Voltaje_FA',	'Voltaje_FC',	'Outlier_Active_energy',	'Outlier_Reactive_energy',	'Outlier_Voltaje_FA',	'Outlier_Voltaje_FC'], axis=1)
Anomalias_2.loc[Anomalias_2.Anomalia>0, 'Anomalia']=1
Anomalias_2

,fuente,Year,Month,Day,Anomalia
0,1,2021,1,1,0
1,1,2021,1,2,0
2,1,2021,1,3,0
3,1,2021,1,4,0
4,1,2021,1,5,0
...,...,...,...,...,...
19312,9,2023,3,27,0
19313,9,2023,3,28,0
19314,9,2023,3,29,0
19315,9,2023,3,30,0


In [56]:
datos_2 = outlier_df2.drop([ 'Outlier_Active_energy','Outlier_Reactive_energy', 'Outlier_Voltaje_FA', 'Outlier_Voltaje_FC',
       'Anomalia'], axis=1)
datos_2

,fuente,Year,Month,Day,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
0,1,2021,1,1,29.972064,15.233026,11624.168389,11637.857473
1,1,2021,1,2,36.978240,18.194945,12208.163321,11936.657816
2,1,2021,1,3,34.909377,18.749972,12033.759791,12881.126615
3,1,2021,1,4,36.647375,19.020038,11945.363519,12493.292822
4,1,2021,1,5,33.051524,17.881993,12020.001492,11668.700439
...,...,...,...,...,...,...,...,...
19312,9,2023,3,27,5.371967,8.267639,26241.124056,26762.891164
19313,9,2023,3,28,5.910494,8.670036,25390.551163,25555.142219
19314,9,2023,3,29,6.022898,8.333407,29584.937317,28882.951142
19315,9,2023,3,30,5.459185,8.339315,28592.978517,30261.498515


In [57]:
# Configurando el Isolation Forest Agrupación Año Mes y dia
clf2 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                      max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

# Entrenar el Isolation Forest
clf2.fit(datos_2)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds2 = clf2.predict(datos_2)

# Añadir las predicciones al DataFrame
datos_2['anomaly'] = preds2


In [58]:
anomalies_2 = datos_2[datos_2['anomaly'] == -1]
print(anomalies_2)

      fuente  Year  Month  Day  Active_energy  Reactive_energy    Voltaje_FA  \
28         1  2021      1   29      64.204485         4.921432  11700.525016   
29         1  2021      1   30      66.944749         4.681936  11757.866023   
30         1  2021      1   31      68.000868         4.148977  12025.424479   
244        1  2021      9    2     112.195455        46.626499  12012.768368   
245        1  2021      9    3     134.335195        56.785614  11506.081241   
...      ...   ...    ...  ...            ...              ...           ...   
19305      9  2023      3   20       5.103524         8.355096   2640.326215   
19306      9  2023      3   21       6.235465         8.665501   3681.050448   
19307      9  2023      3   22       4.752891         8.671021   4231.995740   
19308      9  2023      3   23       6.253955         8.322992   6224.049721   
19316      9  2023      3   31       5.257825         8.521472  29491.566469   

         Voltaje_FC  anomaly  
28     1

In [67]:
Anomalias_IF2 = anomalies_2.drop(['Active_energy','Reactive_energy','Voltaje_FA',	'Voltaje_FC'], axis=1)
Validacion_2=Anomalias_2.merge(Anomalias_IF2,how='outer')
Validacion_2['anomaly'] = Validacion_2['anomaly'].fillna(0)
Validacion_2


,fuente,Year,Month,Day,Anomalia,anomaly
0,1,2021,1,1,0,0.0
1,1,2021,1,2,0,0.0
2,1,2021,1,3,0,0.0
3,1,2021,1,4,0,0.0
4,1,2021,1,5,0,0.0
...,...,...,...,...,...,...
19312,9,2023,3,27,0,0.0
19313,9,2023,3,28,0,0.0
19314,9,2023,3,29,0,0.0
19315,9,2023,3,30,0,0.0


In [68]:
Validacion_2['resultado']=Validacion_2['Anomalia']+Validacion_2['anomaly']
Validacion_2.groupby(['fuente','Year', 'Month'])[['Anomalia',	'anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)
Validacion_2.head()

,fuente,Year,Month,Day,Anomalia,anomaly,resultado
0,1,2021,1,1,0,0.0,0.0
1,1,2021,1,2,0,0.0,0.0
2,1,2021,1,3,0,0.0,0.0
3,1,2021,1,4,0,0.0,0.0
4,1,2021,1,5,0,0.0,0.0


In [69]:
tabla_cruzada_2 = pd.crosstab(Validacion_2['Anomalia'], Validacion_2['anomaly'])
print(tabla_cruzada_2)


anomaly   -1.0    0.0
Anomalia             
0         2120  16695
1          198    304


AGRUPACIÓN POR AÑO MES DIA y HORA (3)

In [72]:
SB_Agrupado3 = SB.groupby(["fuente", "Year","Month","Day","Hour"])[["fuente", "Year","Month","Day","Hour","Active_energy","Reactive_energy","Voltaje_FA","Voltaje_FC"]].agg({
    'fuente':'first',
    'Year':'first',
    'Month':'first',
    'Day':'first',
    'Hour':'first',
    "Active_energy":'sum',
    "Reactive_energy":'sum',
    "Voltaje_FA":'sum',
    "Voltaje_FC":'sum'
    }
)
SB_Agrupado3.describe()


,Year,Month,Day,Hour,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
count,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000
mean,2021.500912,5.955615,15.607762,11.497209,1.472884,0.873086,1420.188470,1438.515836
std,0.621279,3.519334,8.783858,6.922158,1.717838,1.158846,766.299118,746.447449
min,2021.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.031000,0.031000
25%,2021.000000,3.000000,8.000000,5.000000,0.242788,0.112832,719.462874,748.148254
50%,2021.000000,6.000000,16.000000,11.000000,0.810771,0.380650,1625.493463,1634.696089
75%,2022.000000,9.000000,23.000000,17.000000,1.992488,1.222834,2037.276385,2040.554497
max,2023.000000,12.000000,31.000000,23.000000,14.622644,11.135141,4266.229746,4399.038932


In [73]:
SB_Agrupado3 = SB_Agrupado3[~((SB_Agrupado3['Year']==2023) & (SB_Agrupado3['Month']==4))]
SB_Agrupado3=SB_Agrupado3.reset_index(drop=True)


In [74]:
outlier_df3 = SB_Agrupado3.groupby('fuente').apply(identify_outliers)

In [75]:
outlier_df3['Anomalia']= (outlier_df3['Outlier_Active_energy']+outlier_df3['Outlier_Reactive_energy']+outlier_df3['Outlier_Voltaje_FA']+outlier_df3['Outlier_Voltaje_FC'])
outlier_df3=outlier_df3.reset_index(drop=True)
outlier_df3.groupby('fuente')[['Anomalia']].agg("sum").sort_values(by ='Anomalia', ascending=False)

,Anomalia
fuente,
DATOSCLIENTE4.csv,3036
DATOSCLIENTE2.csv,2779
DATOSCLIENTE3.csv,2613
DATOSCLIENTE5.csv,1853
DATOSCLIENTE1.csv,1817
DATOSCLIENTE14.csv,1403
DATOSCLIENTE8.csv,1219
DATOSCLIENTE19.csv,1119
DATOSCLIENTE20.csv,975


In [76]:
outlier_df3['fuente']=outlier_df3['fuente'].str.replace('DATOSCLIENTE','').str.replace('.csv','')
outlier_df3

,fuente,Year,Month,Day,Hour,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Outlier_Active_energy,Outlier_Reactive_energy,Outlier_Voltaje_FA,Outlier_Voltaje_FC,Anomalia
0,1,2021,1,1,0,0.357841,0.282788,455.139171,510.561002,0,0,0,0,0
1,1,2021,1,1,1,0.372264,0.431377,469.978787,469.917178,0,0,0,0,0
2,1,2021,1,1,2,1.044687,0.338626,468.721120,546.949147,0,0,0,0,0
3,1,2021,1,1,3,0.566425,0.495791,452.329255,444.122989,0,0,0,0,0
4,1,2021,1,1,4,1.080556,0.472018,513.477596,535.463719,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463410,9,2023,3,31,19,0.138272,0.296173,1414.666170,1354.851565,0,0,0,0,0
463411,9,2023,3,31,20,0.231000,0.309493,1343.358528,1288.567372,0,0,0,0,0
463412,9,2023,3,31,21,0.041937,0.314091,1050.653622,1266.553989,0,0,0,0,0
463413,9,2023,3,31,22,0.231000,0.266937,1104.985834,991.259378,0,0,0,0,0


In [77]:
Anomalias_3= outlier_df3.drop(['Active_energy','Reactive_energy','Voltaje_FA', 'Voltaje_FC',   'Outlier_Active_energy',    'Outlier_Reactive_energy',  'Outlier_Voltaje_FA',   'Outlier_Voltaje_FC'], axis=1)
Anomalias_3.loc[Anomalias_3.Anomalia>0, 'Anomalia']=1
Anomalias_3

,fuente,Year,Month,Day,Hour,Anomalia
0,1,2021,1,1,0,0
1,1,2021,1,1,1,0
2,1,2021,1,1,2,0
3,1,2021,1,1,3,0
4,1,2021,1,1,4,0
...,...,...,...,...,...,...
463410,9,2023,3,31,19,0
463411,9,2023,3,31,20,0
463412,9,2023,3,31,21,0
463413,9,2023,3,31,22,0


In [78]:
datos_3 = outlier_df3.drop([ 'Outlier_Active_energy','Outlier_Reactive_energy', 'Outlier_Voltaje_FA', 'Outlier_Voltaje_FC',
       'Anomalia'], axis=1)
datos_3

,fuente,Year,Month,Day,Hour,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
0,1,2021,1,1,0,0.357841,0.282788,455.139171,510.561002
1,1,2021,1,1,1,0.372264,0.431377,469.978787,469.917178
2,1,2021,1,1,2,1.044687,0.338626,468.721120,546.949147
3,1,2021,1,1,3,0.566425,0.495791,452.329255,444.122989
4,1,2021,1,1,4,1.080556,0.472018,513.477596,535.463719
...,...,...,...,...,...,...,...,...,...
463410,9,2023,3,31,19,0.138272,0.296173,1414.666170,1354.851565
463411,9,2023,3,31,20,0.231000,0.309493,1343.358528,1288.567372
463412,9,2023,3,31,21,0.041937,0.314091,1050.653622,1266.553989
463413,9,2023,3,31,22,0.231000,0.266937,1104.985834,991.259378


In [79]:
# Configurando el Isolation Forest Agrupación Año Mes dia y hora
clf3 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                      max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

# Entrenar el Isolation Forest
clf3.fit(datos_3)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds3 = clf3.predict(datos_3)

# Añadir las predicciones al DataFrame
datos_3['anomaly'] = preds3

anomalies_3 = datos_3[datos_3['anomaly'] == -1]
print(anomalies_3)


       fuente  Year  Month  Day  Hour  Active_energy  Reactive_energy  \
0           1  2021      1    1     0       0.357841         0.282788   
1           1  2021      1    1     1       0.372264         0.431377   
2           1  2021      1    1     2       1.044687         0.338626   
23          1  2021      1    1    23       1.431520         0.739584   
24          1  2021      1    2     0       1.557737         0.995779   
...       ...   ...    ...  ...   ...            ...              ...   
463410      9  2023      3   31    19       0.138272         0.296173   
463411      9  2023      3   31    20       0.231000         0.309493   
463412      9  2023      3   31    21       0.041937         0.314091   
463413      9  2023      3   31    22       0.231000         0.266937   
463414      9  2023      3   31    23       0.150296         0.312547   

         Voltaje_FA   Voltaje_FC  anomaly  
0        455.139171   510.561002       -1  
1        469.978787   469.917178   

In [80]:
Anomalias_IF3 = anomalies_3.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_3=Anomalias_3.merge(Anomalias_IF3,how='outer')
Validacion_3['anomaly'] = Validacion_3['anomaly'].fillna(0)
Validacion_3


,fuente,Year,Month,Day,Hour,Anomalia,anomaly
0,1,2021,1,1,0,0,-1.0
1,1,2021,1,1,1,0,-1.0
2,1,2021,1,1,2,0,-1.0
3,1,2021,1,1,3,0,0.0
4,1,2021,1,1,4,0,0.0
...,...,...,...,...,...,...,...
463410,9,2023,3,31,19,0,-1.0
463411,9,2023,3,31,20,0,-1.0
463412,9,2023,3,31,21,0,-1.0
463413,9,2023,3,31,22,0,-1.0


In [81]:
Validacion_3['resultado']=Validacion_3['Anomalia']+Validacion_3['anomaly']
Validacion_3.groupby(['fuente','Year', 'Month'])[['Anomalia',   'anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)
Validacion_3.head()


,fuente,Year,Month,Day,Hour,Anomalia,anomaly,resultado
0,1,2021,1,1,0,0,-1.0,-1.0
1,1,2021,1,1,1,0,-1.0,-1.0
2,1,2021,1,1,2,0,-1.0,-1.0
3,1,2021,1,1,3,0,0.0,0.0
4,1,2021,1,1,4,0,0.0,0.0


In [82]:
tabla_cruzada_3 = pd.crosstab(Validacion_3['Anomalia'], Validacion_3['anomaly'])
print(tabla_cruzada_3)


anomaly    -1.0     0.0
Anomalia               
0         49974  395789
1          5636   12016


AGRUPACIÓN POR AÑO MES DIA DIA DE LA SEMANA y HORA (4)

In [83]:
SB_Agrupado4 = SB.groupby(["fuente", "Year","Month","Day","DayWeek","Hour"])[["fuente", "Year","Month","Day","DayWeek","Hour","Active_energy","Reactive_energy","Voltaje_FA","Voltaje_FC"]].agg({
    'fuente':'first',
    'Year':'first',
    'Month':'first',
    'Day':'first',
    'DayWeek':'first',
    'Hour':'first',
    "Active_energy":'sum',
    "Reactive_energy":'sum',
    "Voltaje_FA":'sum',
    "Voltaje_FC":'sum'
    }
)
SB_Agrupado4.describe()


,Year,Month,Day,DayWeek,Hour,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
count,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000,463425.000000
mean,2021.500912,5.955615,15.607762,3.001746,11.497209,1.472884,0.873086,1420.188470,1438.515836
std,0.621279,3.519334,8.783858,1.999393,6.922158,1.717838,1.158846,766.299118,746.447449
min,2021.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.031000,0.031000
25%,2021.000000,3.000000,8.000000,1.000000,5.000000,0.242788,0.112832,719.462874,748.148254
50%,2021.000000,6.000000,16.000000,3.000000,11.000000,0.810771,0.380650,1625.493463,1634.696089
75%,2022.000000,9.000000,23.000000,5.000000,17.000000,1.992488,1.222834,2037.276385,2040.554497
max,2023.000000,12.000000,31.000000,6.000000,23.000000,14.622644,11.135141,4266.229746,4399.038932


In [84]:
SB_Agrupado4 = SB_Agrupado4[~((SB_Agrupado4['Year']==2023) & (SB_Agrupado4['Month']==4))]
SB_Agrupado4=SB_Agrupado4.reset_index(drop=True)

In [87]:
outlier_df4= SB_Agrupado4.groupby('fuente').apply(identify_outliers)

In [88]:
outlier_df4['Anomalia']= (outlier_df4['Outlier_Active_energy']+outlier_df4['Outlier_Reactive_energy']+outlier_df4['Outlier_Voltaje_FA']+outlier_df4['Outlier_Voltaje_FC'])
outlier_df4=outlier_df4.reset_index(drop=True)
outlier_df4.groupby('fuente')[['Anomalia']].agg("sum").sort_values(by ='Anomalia', ascending=False)


,Anomalia
fuente,
DATOSCLIENTE4.csv,3036
DATOSCLIENTE2.csv,2779
DATOSCLIENTE3.csv,2613
DATOSCLIENTE5.csv,1853
DATOSCLIENTE1.csv,1817
DATOSCLIENTE14.csv,1403
DATOSCLIENTE8.csv,1219
DATOSCLIENTE19.csv,1119
DATOSCLIENTE20.csv,975


In [89]:
outlier_df4['fuente']=outlier_df4['fuente'].str.replace('DATOSCLIENTE','').str.replace('.csv','')
outlier_df4

,fuente,Year,Month,Day,DayWeek,Hour,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Outlier_Active_energy,Outlier_Reactive_energy,Outlier_Voltaje_FA,Outlier_Voltaje_FC,Anomalia
0,1,2021,1,1,4,0,0.357841,0.282788,455.139171,510.561002,0,0,0,0,0
1,1,2021,1,1,4,1,0.372264,0.431377,469.978787,469.917178,0,0,0,0,0
2,1,2021,1,1,4,2,1.044687,0.338626,468.721120,546.949147,0,0,0,0,0
3,1,2021,1,1,4,3,0.566425,0.495791,452.329255,444.122989,0,0,0,0,0
4,1,2021,1,1,4,4,1.080556,0.472018,513.477596,535.463719,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463410,9,2023,3,31,4,19,0.138272,0.296173,1414.666170,1354.851565,0,0,0,0,0
463411,9,2023,3,31,4,20,0.231000,0.309493,1343.358528,1288.567372,0,0,0,0,0
463412,9,2023,3,31,4,21,0.041937,0.314091,1050.653622,1266.553989,0,0,0,0,0
463413,9,2023,3,31,4,22,0.231000,0.266937,1104.985834,991.259378,0,0,0,0,0


In [90]:
Anomalias_4 = outlier_df4.drop(['Active_energy','Reactive_energy','Voltaje_FA', 'Voltaje_FC',   'Outlier_Active_energy',    'Outlier_Reactive_energy',  'Outlier_Voltaje_FA',   'Outlier_Voltaje_FC'], axis=1)
Anomalias_4.loc[Anomalias_4.Anomalia>0, 'Anomalia']=1
Anomalias_4

,fuente,Year,Month,Day,DayWeek,Hour,Anomalia
0,1,2021,1,1,4,0,0
1,1,2021,1,1,4,1,0
2,1,2021,1,1,4,2,0
3,1,2021,1,1,4,3,0
4,1,2021,1,1,4,4,0
...,...,...,...,...,...,...,...
463410,9,2023,3,31,4,19,0
463411,9,2023,3,31,4,20,0
463412,9,2023,3,31,4,21,0
463413,9,2023,3,31,4,22,0


In [91]:
datos_4 = outlier_df4.drop([ 'Outlier_Active_energy','Outlier_Reactive_energy', 'Outlier_Voltaje_FA', 'Outlier_Voltaje_FC',
       'Anomalia'], axis=1)
datos_4

,fuente,Year,Month,Day,DayWeek,Hour,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC
0,1,2021,1,1,4,0,0.357841,0.282788,455.139171,510.561002
1,1,2021,1,1,4,1,0.372264,0.431377,469.978787,469.917178
2,1,2021,1,1,4,2,1.044687,0.338626,468.721120,546.949147
3,1,2021,1,1,4,3,0.566425,0.495791,452.329255,444.122989
4,1,2021,1,1,4,4,1.080556,0.472018,513.477596,535.463719
...,...,...,...,...,...,...,...,...,...,...
463410,9,2023,3,31,4,19,0.138272,0.296173,1414.666170,1354.851565
463411,9,2023,3,31,4,20,0.231000,0.309493,1343.358528,1288.567372
463412,9,2023,3,31,4,21,0.041937,0.314091,1050.653622,1266.553989
463413,9,2023,3,31,4,22,0.231000,0.266937,1104.985834,991.259378


In [92]:
# Configurando el Isolation Forest Agrupación Año Mes dia DIA SEMANA Y HORA
clf4 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                      max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

# Entrenar el Isolation Forest
clf4.fit(datos_4)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds4 = clf4.predict(datos_4)

# Añadir las predicciones al DataFrame
datos_4['anomaly'] = preds4

anomalies_4 = datos_4[datos_4['anomaly'] == -1]
print(anomalies_4)


       fuente  Year  Month  Day  DayWeek  Hour  Active_energy  \
48          1  2021      1    3        6     0       1.390026   
70          1  2021      1    3        6    22       1.401228   
71          1  2021      1    3        6    23       1.936444   
72          1  2021      1    4        0     0       1.347389   
73          1  2021      1    4        0     1       1.236223   
...       ...   ...    ...  ...      ...   ...            ...   
463344      9  2023      3   29        2     1       0.596294   
463366      9  2023      3   29        2    23       0.163945   
463367      9  2023      3   30        3     0       0.261007   
463390      9  2023      3   30        3    23       0.231000   
463391      9  2023      3   31        4     0       0.336112   

        Reactive_energy   Voltaje_FA   Voltaje_FC  anomaly  
48             0.825711   491.102433   608.564544       -1  
70             0.886410   517.349137   479.913671       -1  
71             0.913775   482.797492

In [93]:
Anomalias_IF4 = anomalies_4.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_4=Anomalias_4.merge(Anomalias_IF4,how='outer')
Validacion_4['anomaly'] = Validacion_4['anomaly'].fillna(0)
Validacion_4

,fuente,Year,Month,Day,DayWeek,Hour,Anomalia,anomaly
0,1,2021,1,1,4,0,0,0.0
1,1,2021,1,1,4,1,0,0.0
2,1,2021,1,1,4,2,0,0.0
3,1,2021,1,1,4,3,0,0.0
4,1,2021,1,1,4,4,0,0.0
...,...,...,...,...,...,...,...,...
463410,9,2023,3,31,4,19,0,0.0
463411,9,2023,3,31,4,20,0,0.0
463412,9,2023,3,31,4,21,0,0.0
463413,9,2023,3,31,4,22,0,0.0


In [94]:
Validacion_4['resultado']=Validacion_4['Anomalia']+Validacion_4['anomaly']
Validacion_4.groupby(['fuente','Year', 'Month'])[['Anomalia',   'anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)
Validacion_4.head()

,fuente,Year,Month,Day,DayWeek,Hour,Anomalia,anomaly,resultado
0,1,2021,1,1,4,0,0,0.0,0.0
1,1,2021,1,1,4,1,0,0.0,0.0
2,1,2021,1,1,4,2,0,0.0,0.0
3,1,2021,1,1,4,3,0,0.0,0.0
4,1,2021,1,1,4,4,0,0.0,0.0


In [95]:
tabla_cruzada_4 = pd.crosstab(Validacion_4['Anomalia'], Validacion_4['anomaly'])
print(tabla_cruzada_4)

anomaly    -1.0     0.0
Anomalia               
0         50435  395328
1          5175   12477


modelo (3) diferentes parametros

In [96]:
# Configurando el Isolation Forest Agrupación Año Mes dia y hora
clf5 = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', \
                      max_features=1.0, bootstrap=False, n_jobs= None, random_state=None, verbose=0)

# Entrenar el Isolation Forest
clf5.fit(datos_3)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds5 = clf5.predict(datos_3)

# Añadir las predicciones al DataFrame
datos_3['anomaly'] = preds5

anomalies_5 = datos_3[datos_3['anomaly'] == -1]
print(anomalies_5)

       fuente  Year  Month  Day  Hour  Active_energy  Reactive_energy  \
0           1  2021      1    1     0       0.357841         0.282788   
1           1  2021      1    1     1       0.372264         0.431377   
2           1  2021      1    1     2       1.044687         0.338626   
3           1  2021      1    1     3       0.566425         0.495791   
20          1  2021      1    1    20       1.535616         0.613046   
...       ...   ...    ...  ...   ...            ...              ...   
463410      9  2023      3   31    19       0.138272         0.296173   
463411      9  2023      3   31    20       0.231000         0.309493   
463412      9  2023      3   31    21       0.041937         0.314091   
463413      9  2023      3   31    22       0.231000         0.266937   
463414      9  2023      3   31    23       0.150296         0.312547   

         Voltaje_FA   Voltaje_FC  anomaly  
0        455.139171   510.561002       -1  
1        469.978787   469.917178   

In [97]:
Anomalias_IF5 = anomalies_5.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_5=Anomalias_3.merge(Anomalias_IF5,how='outer')
Validacion_5['anomaly'] = Validacion_5['anomaly'].fillna(0)
Validacion_5

,fuente,Year,Month,Day,Hour,Anomalia,anomaly
0,1,2021,1,1,0,0,-1.0
1,1,2021,1,1,1,0,-1.0
2,1,2021,1,1,2,0,-1.0
3,1,2021,1,1,3,0,-1.0
4,1,2021,1,1,4,0,0.0
...,...,...,...,...,...,...,...
463410,9,2023,3,31,19,0,-1.0
463411,9,2023,3,31,20,0,-1.0
463412,9,2023,3,31,21,0,-1.0
463413,9,2023,3,31,22,0,-1.0


In [98]:
Validacion_5['resultado']=Validacion_5['Anomalia']+Validacion_5['anomaly']
Validacion_5.groupby(['fuente','Year', 'Month'])[['Anomalia',   'anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)
Validacion_5.head()

,fuente,Year,Month,Day,Hour,Anomalia,anomaly,resultado
0,1,2021,1,1,0,0,-1.0,-1.0
1,1,2021,1,1,1,0,-1.0,-1.0
2,1,2021,1,1,2,0,-1.0,-1.0
3,1,2021,1,1,3,0,-1.0,-1.0
4,1,2021,1,1,4,0,0.0,0.0


In [99]:
tabla_cruzada_5 = pd.crosstab(Validacion_5['Anomalia'], Validacion_5['anomaly'])
print(tabla_cruzada_5)

anomaly     -1.0     0.0
Anomalia                
0         119348  326415
1           7786    9866


In [100]:
# Configurando el Isolation Forest Agrupación Año Mes dia y hora
clf6 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.04), \
                      max_features=1.0, bootstrap=False, n_jobs= None, random_state=None, verbose=0)

# Entrenar el Isolation Forest
clf6.fit(datos_3)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds6 = clf6.predict(datos_3)

# Añadir las predicciones al DataFrame
datos_3['anomaly'] = preds6

anomalies_6 = datos_3[datos_3['anomaly'] == -1]
print(anomalies_6)

       fuente  Year  Month  Day  Hour  Active_energy  Reactive_energy  \
1           1  2021      1    1     1       0.372264         0.431377   
717         1  2021      1   30    21       3.259431         0.055036   
719         1  2021      1   30    23       2.920958         0.231000   
720         1  2021      1   31     0       2.773075         0.006843   
743         1  2021      1   31    23       2.898785         0.231000   
...       ...   ...    ...  ...   ...            ...              ...   
463220      9  2023      3   23    21       0.231000         0.269375   
463222      9  2023      3   23    23       0.231000         0.353278   
463232      9  2023      3   24     9       0.231000         0.377390   
463233      9  2023      3   24    10       0.040641         0.331031   
463414      9  2023      3   31    23       0.150296         0.312547   

         Voltaje_FA   Voltaje_FC  anomaly  
1        469.978787   469.917178       -1  
717      505.255465   347.520138   

In [101]:
Anomalias_IF6 = anomalies_6.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_6=Anomalias_3.merge(Anomalias_IF6,how='outer')
Validacion_6['anomaly'] = Validacion_6['anomaly'].fillna(0)
Validacion_6['resultado']=Validacion_6['Anomalia']+Validacion_6['anomaly']
Validacion_6.groupby(['fuente','Year', 'Month'])[['Anomalia','anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)

tabla_cruzada_6 = pd.crosstab(Validacion_6['Anomalia'], Validacion_6['anomaly'])
print(tabla_cruzada_6)


anomaly    -1.0     0.0
Anomalia               
0         16977  428786
1          1560   16092


In [103]:
# Configurando el Isolation Forest Agrupación Año Mes dia y hora
clf7 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.5), \
                      max_features=1.0, bootstrap=False, n_jobs= None, random_state=None, verbose=0)

# Entrenar el Isolation Forest
clf7.fit(datos_3)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds7 = clf7.predict(datos_3)

# Añadir las predicciones al DataFrame
datos_3['anomaly'] = preds7

anomalies_7 = datos_3[datos_3['anomaly'] == -1]
print(anomalies_7)

       fuente  Year  Month  Day  Hour  Active_energy  Reactive_energy  \
0           1  2021      1    1     0       0.357841         0.282788   
1           1  2021      1    1     1       0.372264         0.431377   
2           1  2021      1    1     2       1.044687         0.338626   
3           1  2021      1    1     3       0.566425         0.495791   
4           1  2021      1    1     4       1.080556         0.472018   
...       ...   ...    ...  ...   ...            ...              ...   
463410      9  2023      3   31    19       0.138272         0.296173   
463411      9  2023      3   31    20       0.231000         0.309493   
463412      9  2023      3   31    21       0.041937         0.314091   
463413      9  2023      3   31    22       0.231000         0.266937   
463414      9  2023      3   31    23       0.150296         0.312547   

         Voltaje_FA   Voltaje_FC  anomaly  
0        455.139171   510.561002       -1  
1        469.978787   469.917178   

In [104]:
Anomalias_IF7 = anomalies_7.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_7=Anomalias_3.merge(Anomalias_IF7,how='outer')
Validacion_7['anomaly'] = Validacion_7['anomaly'].fillna(0)
Validacion_7['resultado']=Validacion_7['Anomalia']+Validacion_7['anomaly']
Validacion_7.groupby(['fuente','Year', 'Month'])[['Anomalia','anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)

tabla_cruzada_7 = pd.crosstab(Validacion_7['Anomalia'], Validacion_7['anomaly'])
print(tabla_cruzada_7)

anomaly     -1.0     0.0
Anomalia                
0         220691  225072
1          11016    6636


In [105]:
# Configurando el Isolation Forest Agrupación Año Mes dia y hora
clf8 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.04), \
                      max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

# Entrenar el Isolation Forest
clf8.fit(datos_3)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds8 = clf8.predict(datos_3)

# Añadir las predicciones al DataFrame
datos_3['anomaly'] = preds8

anomalies_8 = datos_3[datos_3['anomaly'] == -1]
print(anomalies_8)

       fuente  Year  Month  Day  Hour  Active_energy  Reactive_energy  \
2136        1  2021      3   31     0       3.535730         1.877968   
5855        1  2021      9    1    23       5.199952         2.277318   
5880        1  2021      9    3     0       4.943343         2.115842   
5900        1  2021      9    3    20       5.963940         2.490716   
5901        1  2021      9    3    21       6.204176         2.564501   
...       ...   ...    ...  ...   ...            ...              ...   
463198      9  2023      3   22    23       0.013140         0.372977   
463222      9  2023      3   23    23       0.231000         0.353278   
463367      9  2023      3   30     0       0.261007         0.385318   
463391      9  2023      3   31     0       0.336112         0.432630   
463414      9  2023      3   31    23       0.150296         0.312547   

         Voltaje_FA   Voltaje_FC  anomaly  
2136     516.902118   420.465060       -1  
5855     504.491260   493.287739   

In [106]:
Anomalias_IF8 = anomalies_8.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_8=Anomalias_3.merge(Anomalias_IF8,how='outer')
Validacion_8['anomaly'] = Validacion_8['anomaly'].fillna(0)
Validacion_8['resultado']=Validacion_8['Anomalia']+Validacion_8['anomaly']
Validacion_8.groupby(['fuente','Year', 'Month'])[['Anomalia','anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)

tabla_cruzada_8 = pd.crosstab(Validacion_8['Anomalia'], Validacion_8['anomaly'])
print(tabla_cruzada_8)

anomaly    -1.0     0.0
Anomalia               
0         16420  429343
1          2117   15535


In [109]:
# Configurando el Isolation Forest Agrupación Año Mes dia y hora
clf9 = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                      max_features=1.0, bootstrap=False, n_jobs=-1, random_state=80, verbose=0)

# Entrenar el Isolation Forest
clf9.fit(datos_3)

# Predecir si una observación es una anomalía. -1 indica anomalía, 1 indica normal.
preds9 = clf9.predict(datos_3)

# Añadir las predicciones al DataFrame
datos_3['anomaly'] = preds9

anomalies_9 = datos_3[datos_3['anomaly'] == -1]
print(anomalies_9)


       fuente  Year  Month  Day  Hour  Active_energy  Reactive_energy  \
0           1  2021      1    1     0       0.357841         0.282788   
23          1  2021      1    1    23       1.431520         0.739584   
24          1  2021      1    2     0       1.557737         0.995779   
46          1  2021      1    2    22       1.587140         0.952514   
47          1  2021      1    2    23       1.746916         0.763222   
...       ...   ...    ...  ...   ...            ...              ...   
463409      9  2023      3   31    18       0.008047         0.313382   
463411      9  2023      3   31    20       0.231000         0.309493   
463412      9  2023      3   31    21       0.041937         0.314091   
463413      9  2023      3   31    22       0.231000         0.266937   
463414      9  2023      3   31    23       0.150296         0.312547   

         Voltaje_FA   Voltaje_FC  anomaly  
0        455.139171   510.561002       -1  
23       493.418529   463.033345   

In [110]:
Anomalias_IF9 = anomalies_9.drop(['Active_energy','Reactive_energy','Voltaje_FA',   'Voltaje_FC'], axis=1)
Validacion_9=Anomalias_3.merge(Anomalias_IF9,how='outer')
Validacion_9['anomaly'] = Validacion_9['anomaly'].fillna(0)
Validacion_9['resultado']=Validacion_9['Anomalia']+Validacion_9['anomaly']
Validacion_9.groupby(['fuente','Year', 'Month'])[['Anomalia','anomaly','resultado']].agg("sum").sort_values(by ='resultado', ascending=False)

tabla_cruzada_9 = pd.crosstab(Validacion_8['Anomalia'], Validacion_8['anomaly'])
print(tabla_cruzada_9)

anomaly    -1.0     0.0
Anomalia               
0         16420  429343
1          2117   15535


In [112]:
Anmalias_KMEANS = pd.read_csv('Datos Anomalias.csv')
Anmalias_KMEANS

,Unnamed: 0,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Cliente,Año,Mes,Trimestre,Dia,Hora,Dia_Semana,anomalias,anomalias_ocsvm,Outlier_Active_energy,Outlier_Reactive_energy,Outlier_Voltaje_FA,Outlier_Voltaje_FC,outlier
0,0,2021-01-01 00:00:00,0.357841,0.282788,455.139171,510.561002,Cliente 1,2021,1,1,1,0,Friday,0,0,0,0,0,0,0
1,1,2021-01-01 01:00:00,0.372264,0.431377,469.978787,469.917178,Cliente 1,2021,1,1,1,1,Friday,0,0,0,0,0,0,0
2,2,2021-01-01 02:00:00,1.044687,0.338626,468.721120,546.949147,Cliente 1,2021,1,1,1,2,Friday,0,0,0,0,0,0,0
3,3,2021-01-01 03:00:00,0.566425,0.495791,452.329255,444.122989,Cliente 1,2021,1,1,1,3,Friday,0,0,0,0,0,0,0
4,4,2021-01-01 04:00:00,1.080556,0.472018,513.477596,535.463719,Cliente 1,2021,1,1,1,4,Friday,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463410,463419,2023-03-31 19:00:00,0.138272,0.296173,1414.666170,1354.851565,Cliente 9,2023,3,1,31,19,Friday,0,0,0,0,0,0,0
463411,463420,2023-03-31 20:00:00,0.231000,0.309493,1343.358528,1288.567372,Cliente 9,2023,3,1,31,20,Friday,0,0,0,0,0,0,0
463412,463421,2023-03-31 21:00:00,0.041937,0.314091,1050.653622,1266.553989,Cliente 9,2023,3,1,31,21,Friday,0,0,0,0,0,0,0
463413,463422,2023-03-31 22:00:00,0.231000,0.266937,1104.985834,991.259378,Cliente 9,2023,3,1,31,22,Friday,0,0,0,0,0,0,0


In [113]:
anomaliasK = Anmalias_KMEANS.drop(['Fecha',	'Active_energy','Reactive_energy','Voltaje_FA',	'Voltaje_FC'], axis=1)

In [ ]:

anomaliasT = Validacion_9.merge(anomaliasK,how='outer')

# Graficas Series de tiempo

Se hace uso de la libreria Prophet, especializada en el analisis de series de tiempo, permite encontrar caracteristicas de las series y hacer predicciones. En particular para este problema se sacara provecho de la primera parte, encontrar caracteristicas de los datos para identificar patrones de comportamiento.

In [39]:
# Para poder extraer la información, se debe correr un modelo para cada cliente en cada variable. 
#El resultado de todos estos modelos se guarda en un dataframe 

clientes = SB['fuente'].unique()
vari = ["Active_energy", "Reactive_energy", "Voltaje_FA", "Voltaje_FC"]

corridaCompleta = pd.DataFrame()

for ii in clientes:
    for jj in vari:
        tsData = SB.loc[SB['fuente'] == ii, ['Fecha', jj]]
        tsData = tsData.rename(columns={'Fecha': 'ds', jj: 'y'})
        
        serie_model = Prophet()
        serie_model.fit(tsData)
        
        future = serie_model.make_future_dataframe(periods=1)
        forecast = serie_model.predict(future)
                
        forecast['cliente'] = ii
        forecast['variable'] = jj
       
        corridaCompleta = pd.concat([corridaCompleta, forecast], ignore_index=True)
        
        print(ii)
        print(jj)

22:31:04 - cmdstanpy - INFO - Chain [1] start processing
22:31:25 - cmdstanpy - INFO - Chain [1] done processing


DATOSCLIENTE1.csv
Active_energy


22:31:32 - cmdstanpy - INFO - Chain [1] start processing
22:31:52 - cmdstanpy - INFO - Chain [1] done processing


DATOSCLIENTE1.csv
Reactive_energy


22:31:58 - cmdstanpy - INFO - Chain [1] start processing
22:32:31 - cmdstanpy - INFO - Chain [1] done processing


...

DATOSCLIENTE30.csv
Reactive_energy


22:53:48 - cmdstanpy - INFO - Chain [1] start processing
22:53:49 - cmdstanpy - INFO - Chain [1] done processing


DATOSCLIENTE30.csv
Voltaje_FA


22:53:52 - cmdstanpy - INFO - Chain [1] start processing
22:53:52 - cmdstanpy - INFO - Chain [1] done processing


DATOSCLIENTE30.csv
Voltaje_FC


In [ ]:
#empezo 22:31

In [41]:
corridaCompleta

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,cliente,variable
0,2021-01-01 00:00:00,0.679192,0.389673,1.864957,0.679192,0.679192,0.471672,0.471672,0.471672,-0.032488,...,-0.067720,0.571880,0.571880,0.571880,0.0,0.0,0.0,1.150864,DATOSCLIENTE1.csv,Active_energy
1,2021-01-01 01:00:00,0.680636,0.273145,1.791604,0.680636,0.680636,0.347686,0.347686,0.347686,-0.177196,...,-0.047334,0.572216,0.572216,0.572216,0.0,0.0,0.0,1.028323,DATOSCLIENTE1.csv,Active_energy
2,2021-01-01 02:00:00,0.682081,0.282062,1.768617,0.682081,0.682081,0.315329,0.315329,0.315329,-0.230294,...,-0.026918,0.572541,0.572541,0.572541,0.0,0.0,0.0,0.997410,DATOSCLIENTE1.csv,Active_energy
3,2021-01-01 03:00:00,0.683525,0.311934,1.868314,0.683525,0.683525,0.408524,0.408524,0.408524,-0.157616,...,-0.006716,0.572855,0.572855,0.572855,0.0,0.0,0.0,1.092049,DATOSCLIENTE1.csv,Active_energy
4,2021-01-01 04:00:00,0.684970,0.514352,1.986308,0.684970,0.684970,0.562672,0.562672,0.562672,-0.023521,...,0.013035,0.573158,0.573158,0.573158,0.0,0.0,0.0,1.247642,DATOSCLIENTE1.csv,Active_energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853815,2022-04-21 11:00:00,1060.955933,914.449796,1230.565155,1060.955933,1060.955933,5.396128,5.396128,5.396128,2.853578,...,2.542550,NaN,NaN,NaN,0.0,0.0,0.0,1066.352060,DATOSCLIENTE30.csv,Voltaje_FC
1853816,2022-04-21 12:00:00,1060.954909,914.407922,1229.293033,1060.954909,1060.954909,5.723734,5.723734,5.723734,3.538601,...,2.185133,NaN,NaN,NaN,0.0,0.0,0.0,1066.678643,DATOSCLIENTE30.csv,Voltaje_FC
1853817,2022-04-21 13:00:00,1060.953886,905.187972,1225.859285,1060.953886,1060.953886,3.700917,3.700917,3.700917,1.887759,...,1.813157,NaN,NaN,NaN,0.0,0.0,0.0,1064.654802,DATOSCLIENTE30.csv,Voltaje_FC
1853818,2022-04-21 14:00:00,1060.952862,907.143332,1230.820157,1060.952862,1060.952862,0.665178,0.665178,0.665178,-0.764612,...,1.429790,NaN,NaN,NaN,0.0,0.0,0.0,1061.618040,DATOSCLIENTE30.csv,Voltaje_FC


In [124]:
#de la corrida completa de los modelos se extrae la información de tendencia para la gráfica 1

datoTendencia = corridaCompleta[['ds', 'trend', 'cliente', 'variable']]
datoTendencia['x'] = datoTendencia['ds'].dt.strftime("%Y-%m")
datoTendencia = datoTendencia.rename(columns={'trend' : 'valor'}).assign(categoria='Tendencia general')
datoTendencia = datoTendencia[['x', 'valor', 'categoria', 'cliente', 'variable' ]]

C:\Users\ma.neisa\AppData\Local\Temp\ipykernel_9464\1410651024.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datoTendencia['x'] = datoTendencia['ds'].dt.strftime("%Y-%m")


In [137]:
#de la corrida completa de los modelos se extrae la información de tendencia por hora del dia
#se debe hacer un procesamiento para conservar un registro por hora

datoHora = corridaCompleta[['ds', 'daily', 'cliente', 'variable']]
datoHora['x'] = datoHora['ds'].dt.strftime("%H:%M:%S")
datoHora = datoHora.groupby(['cliente', 'variable', 'x'])['daily'].mean().reset_index()

datoHora = datoHora.rename(columns={'daily' : 'valor'}).assign(categoria='Tendencia por hora del día')
datoHora = datoHora[['x', 'valor', 'categoria', 'cliente', 'variable' ]]

C:\Users\ma.neisa\AppData\Local\Temp\ipykernel_9464\605153358.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datoHora['x'] = datoHora['ds'].dt.strftime("%H:%M:%S")


In [131]:
#de la corrida completa de los modelos se extrae la información de tendencia por dia de la semana
#se debe hacer un procesamiento para conservar un registro por dia

datoDia = corridaCompleta[['ds', 'weekly', 'cliente', 'variable']]
datoDia['dia'] = pd.to_datetime(datoDia['ds']).dt.day_name()
datoDia = datoDia.groupby(['cliente', 'variable', 'dia'])['weekly'].mean().reset_index()

datoDia['x'] = datoDia['dia'].replace({"Friday": "viernes", "Monday": "lunes", "Saturday": "sábado", 
                       "Sunday" : "domingo", "Thursday" : "jueves", "Tuesday" : "martes", 
                       "Wednesday" : "miércoles"})

datoDia = datoDia.rename(columns={'weekly' : 'valor'}).assign(categoria='Tendencia por día de la semana')
datoDia = datoDia[['x', 'valor', 'categoria', 'cliente', 'variable' ]]

C:\Users\ma.neisa\AppData\Local\Temp\ipykernel_9464\1909678316.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datoDia['dia'] = pd.to_datetime(datoDia['ds']).dt.day_name()


In [151]:
#Se concatenan los datos anteriores preparando el archivo para guardar
datosTodoTendencia = pd.concat([datoTendencia, datoDia, datoHora])
datosTodoTendencia['cliente']=datosTodoTendencia['cliente'].str.replace('DATOSCLIENTE','Cliente ').str.replace('.csv','')



C:\Users\ma.neisa\AppData\Local\Temp\ipykernel_9464\2817277209.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  datosTodoTendencia['cliente']=datosTodoTendencia['cliente'].str.replace('DATOSCLIENTE','Cliente ').str.replace('.csv','')


In [156]:
datosTodoTendencia.to_csv("datosSeries.csv", index=False)